Este archivo contiene la implementación - Funciones, clases y sus definiciones - de la Reasignación de Frecuencias, para el Premio Innovación Emergente y la tesis de Myt

Importo la clase Elec_Array y otras funciones y objetos

In [14]:
import os
#os.chdir(r'C:\Users\Graciela\Desktop\Cochlear Implants\CODE ultima version')
#run "CIS_6.ipynb"

import import_ipynb
import CIS_6

# Estructuras (Patrón, electrodo, nota, melodía)

In [ ]:
##Attempt 2, más rápido, que ande con implantes de 22 electrodos
import numpy as np
from numpy import log2, sin, cos

#Un electrodo
class Elec:
    def __init__(self, freq, on, i):
        self.freq = freq #Frecuencia en la coclea
        self.on = on
        self.i = i #numero de canal
        #self.dist = None  cents(freq,fo)    

   #Busca el parcial de la nota más cercano a la frecuencia del electrodo 
    def buscarFreqMascercana(self, nota):
        min_index = -1
        min_interval = np.inf
        freqs = nota.parciales

        for i in range(len(freqs)):
            interval = np.abs(cents(self.freq,freqs[i]))
            if interval < min_interval:
                min_interval = interval
                min_index = i

        return freqs[min_index], min_index, min_interval

    def copy(self):
        return Elec(self.freq, self.on, self.i)
    

#Es una lista de electrodos activos con un score
#Antes de v3, tambien guardaba los electrodos apagados
class Patron:
    def __init__(self, elecs): #le paso todos los electrodos, y me quedo solo los activos
        self.elecs = []
        for elec in elecs:
            if (elec.on): #solo guardo los prendidos
                self.elecs.append(elec) #Tengo que crear nuevas instancias porque si no chocan en la función recursiva
        self.score = np.inf #TEMPORAL. Hay muchos patrones que se van a evaluar para una nota. Una vez que se eligió el mejor patron para esa nota, los mismos patrones se vuelven a reevaluar para otras notas, y este valor se reinicia. asi que este es volatil y temporal nomas
        self.N = len(self.elecs) 
        self.repiteParcial = None #me dice si hay dos electrodos que estan representando a un mismo parcial. Util saberlo si pienso separar en los patrones "clases" con N electrodos activados, digo. 

    def print(self):
        for i, elec in enumerate(self.elecs):
            if(not elec.on): continue
            print('e' + str(i+1),':', str(round(elec.freq)) + ' Hz')
            print()

        print('Score: '+ str(np.round(self.score)))
        print()

    def copia(self):
        return Patron(self.elecs.copy())


class Nota: #cada nota está representada por una frecuencia fundamental, una lista con la amplitud de sus parciales, y una duración (en segundos). Puede ser un sonido complejo, con componentes inarmónicas. 
    
    #Si la lista de parciales viene vacía, se asume entonces que es armónica y "amps" tiene la amplitud de los armónicos (multiplos enteros) y que puede tener 0's. Se llenan entonces los parciales y se eliminan los 0's.
    #Si "parciales" no viene vacía, entonces tiene que tener la misma longitud que "amps"
    #desafinaciones viene en "cents" - una por cada parcial, incluyendo f0

    #Todo a futuro: Definir un struct "Parcial" que tenga una freq(t) y una A(t) para hacer esto.
    def __init__(self, f0, amps, dur, parciales = None, amp = 1, vibDepth = 0.15, vibFreq = 1.5, env = None, desafinaciones_cents = None, ti = None, mejorPatron = None, score = None): #"Parciales" tipicamente es "[]""
        if(parciales is not None and len(parciales) != len(amps)): Exception('No coincide la lista de amplitudes con la de parciales')
        
        self.freq = f0 #antes: igual a parciales[0] (si no  hay desafinacion o si  desafinacinoes[0] es 0). Luego de que se condensa ya no
        self.amp = amp #Amplitud global
        if(amps is not None): self.amps = amps.copy() #amplitud de los parciales
        if(parciales is not None): self.parciales = parciales.copy() #Frecuencias de los parciales
        
        self.dur  = dur #en segundos
        
        #Info para la sintesis
        self.vibDepth = vibDepth #en semitonos (hacia arriba y hacia abajo)
        self.vibFreq = vibFreq
        if(env is None): #Seteo automático la envolvente:
            self.envinfo = [self.dur* 0.1*1000, self.dur*0.05 * 1000, 1, self.dur*0.1*1000] #ADSR en ms
        else: self.envinfo = env.copy()
        

        if(desafinaciones_cents is not None):
            if(len(desafinaciones_cents) != len(amps)): Exception('No coincide la lista de amplitudes con la de desafinaciones')
            self.parciales = []
            for i in range(len(desafinaciones_cents)):
                d = desafinaciones_cents[i]
                intervalo = 2**(d/1200)
                f_des = f0*(i+1)*intervalo
                self.parciales.append(f_des)

        #Si la lista de parciales viene vacía, es nota armónica y hago lo que dije arriba
        if(parciales is None):
            self.amps = []
            self.parciales = []
            for i in range(len(amps)):
                if(amps[i] != 0):
                    self.parciales.append(f0*(i+1))
                    self.amps.append(amps[i])

        self.parciales = np.array(self.parciales)
        self.ti = ti #Se asigna después. Para ser usado en el contexto de una melodia

        #aproximacion por electrodos
        if(mejorPatron is not None): self.mejorPatron = mejorPatron.copia() #se asigna despues
        else: self.mejorPatron = None
        self.score = score #se asigna despues. Este es el importante, es el error de aproximacion por el self.mejorPatrons
        return

    def copia(self):
       return Nota(self.freq, self.amps, self.dur, self.parciales, self.amp, self.vibDepth, self.vibFreq, self.envinfo,  ti= self.ti, mejorPatron = self.mejorPatron, score = self.score)


    def transportar(self, intervalo):
        self.freq *= intervalo
        self.parciales = self.parciales * intervalo #es nparray asi que anda bien
        return

    #si tiene 8 parciales y 8 amplitudes tipo [1,0,0,1/2,0,0,1/3,0], deja solo los parciales y amps 0,3,6 
    def condensarParciales(self):
        if(len(self.amps) != len(self.parciales)): Exception(f'Algo esta mal son esta nota: {self.parciales}, {self.amps}')
        parciales2 = []
        amps2 = []
        for i, amp in enumerate(self.amps): 
            if amp != 0: 
                parciales2.append(self.parciales[i])
                amps2.append(amp)
        self.parciales = parciales2
        self.amps = amps2


    #Idem, pero no especifico parametros sino que uso los de la nota misma
    #OJO QUE ANTES DE LA V4 DE ESTE NOTEBOOK, ESTABA MAL ESTO!! aplicaba mal la envolvente!! Boludo... me quedaba todo al doble de frecuencia...
    def sintetizar3(self, sr):
        N = len(self.amps)
        (a,d,s,r) = (self.envinfo[0], self.envinfo[1], self.envinfo[2], self.envinfo[3] ) 

        parciales = [CIS_6.puretoneVib2(self.parciales[i], self.dur, sr, self.vibDepth, self.vibFreq, self.dur*0.3 )*self.amps[i] for i in range(N)]
        suma = 1/N* np.sum( parciales, axis=0)
        #la funcion adsre ya aplica las envolventes a la señal. Pero como tengo varios parciales, en vez de generar la envolvente cada vez puedo aplicarsela a una
        # u[n] unit step y conseguir asi la "envolvente". Si quisiera optimizar-
        suma_con_env = CIS_6.adsre(a,d,s,r, suma, sr)

        return self.amp * suma_con_env
      
    #sintetiza esta nota pero usando noise
    #bw: ancho del filtro en semitonos alrededor de cada parcial
    def sintetizarNoise(self, sr, bw = 1):
        N = len(self.amps)
        (a,d,s,r) = (self.envinfo[0], self.envinfo[1], self.envinfo[2], self.envinfo[3] ) 

        noise = CIS_6.wnoise_sig(round(self.dur*sr))
        out = np.zeros(len(noise))
        orden = 6
        for fc, amp in zip(self.parciales, self.amps):

            f1 = fc / 2**(bw/24)
            f2 = fc * 2**(bw/24)

            out +=  CIS_6.aplicar_filtro(noise, orden, sr, f1, f2 ) * amp
        
        #aplico envolvente
        out = CIS_6.adsre(a,d,s,r, out, sr) 
        out *= self.amp/ N

        return  out

    #Muestro en texto la info
    def printInformeScores(self, outpath = ''):
        armonicos_presentes = [self.parciales[i]/self.freq for i in range(len(self.parciales))]
        print('--- F0 : ', f'{self.freq} Hz --- ', f'Armonicos presentes {np.round(armonicos_presentes, 0)} \n')

        self.repiteParcial = False #Si un parcial es representado por mas de un electrodo
        for i in range(len(self.mejorPatron.elecs)):

            eleci = self.mejorPatron.elecs[i]

            f_armonico_cercano, i_armonico_cercano, _ = eleci.buscarFreqMascercana(self)
            dist = cents(eleci.freq, f_armonico_cercano)

            if(i_armonico_cercano  != i): self.repiteParcial = True 
            n_armonico_cercano = f_armonico_cercano/self.freq #que numero de armonico es, si 3f0, 4f0, etc)


            print(f'{n_armonico_cercano:.0f}° Parcial ', f'({f_armonico_cercano:.0f}Hz)', f'aproximado por canal {eleci.i + 1} de freq {eleci.freq:.0f}Hz')
            print(f'Error: {dist:.0f} cents =  {f_armonico_cercano - eleci.freq} Hz')
        
            print()

        print(f'Score nota {self.score}')
        if(self.repiteParcial):
            print(f'Falta representar un parcial / Un parcial está representado por más de un electrodo')
        else:
            print('Hay correspondendia uno a uno entre parciales y elecs')


#todo que la melodia lleve guardado el parametro F0min F0max N_mayus max_di. O que se logee al guardar los resultados. Pero no lo puedo olvidar ese parametro cuando guardo los resultados...
# y la funcion de score usada
#(Esto en realidad iria en una clase "Experimento", que tiene a todas las melodias puntuadas y estos parametros)
# y la melo original

class Melodia:
    
    #Guardo las notas y sus instantes de inicio
    #Al pasar solo el array de notas, pone una atrás de otra con un pequeño solapamiento especifiacado en milisegundos
    #Al leer el archivo midi, el solapamiento es automático
    def __init__(self, notas, overlap = 0, score = None):
        self.notas = notas
        self.dur = 0

        ti, self.tis = 0, []
        for nota in notas: 
            nota.ti = ti
            ti += nota.dur
            nota.dur += overlap /1000
        if(len(notas) > 1):
            self.dur = ti + self.notas[-1].dur
        else: self.dur = notas[0].dur

        self.score = score #se asigna al ir buscando los patrones
    

    def mejorSecuenciaPatrones(self):
        out = []
        for nota in self.notas: out.append(nota.mejorPatron)
        return out

    def copia(self):
        return Melodia([nota.copia() for nota in self.notas], overlap = 0, score = self.score)

    def asignarAmps(self, amps): #asigna amplitud de parciales a cada nota
        for i, nota in enumerate(self.notas): 
            #nota.amps = amps esto asi solo bugea. uso lo de abajo para regenerar los parciales basicamente
            self.notas[i] = Nota(nota.freq, amps, nota.dur, amp = nota.amp, ti = nota.ti, mejorPatron = nota.mejorPatron, score = nota.score)


    def transportar(self, intervalo): #inplace
        for nota in self.notas: nota.transportar(intervalo)
    
    def calcularScore(self):
        score = 0
        for patron in self.mejorSecuenciaPatrones(): score += patron.score
        self.score = score
        return score

        
    def notasExtremas(self): #Devuelve la f0 más grave y la más aguda
        f_grave = np.inf
        f_aguda = 0
        for nota in self.notas: 
            f_grave = min(f_grave, nota.freq)
            f_aguda = max(f_aguda, nota.freq)
        return f_grave, f_aguda
    

    def leerMidi(path, amps, desafinaciones_cents = None ): #Crea una melodia a partir de un path midi (ojo, se asume que tiene toda la info en el canal 1).
        #Se espera que sea monofónica, pero puede haber overlaps entre las notas. Lo que se va a almacenar son los tiempos de inicio y las duraciones.
        #amps guarda la lista de espectros para cada armónico, que por ahora se va a tomar como constante
        from mido import MidiFile

        mid = MidiFile(path)

        notas = []
        #Información para determinar tiempo absoluto en segundos a partir de los tiempos en clicks del midi
        BPM = None
        PPQ = mid.ticks_per_beat
        ticks2seconds = 0 #Constante para convertir. La seteo abajo al leer el BPM

        nota = None 
        f0, dur, parciales, amp = None, None, None, None
        t_on, t_off = 0,0
        for i, track in enumerate(mid.tracks):
            for msg in track:

                if(msg.type in ['note_on', 'note_off']):
                        
                    if(msg.type == 'note_on' and msg.velocity >0):
                        amp = msg.velocity / 127
                        freq = CIS_6.midi2freq(msg.note)
                        t_on = msg.time
                    
                    #if(msg.type == 'note_off' or msg.velocity == 0):
                    #Note off: Apago la nota
                    else:
                        t_off = msg.time
                        dur = (t_off - t_on) * ticks2seconds 
                        nota = Nota(freq, amps, dur, parciales = None, amp = amp, desafinaciones_cents = desafinaciones_cents)
                        nota.ti = t_on * ticks2seconds
                        notas.append(nota)

                if(msg.type == 'set_tempo'):
                    BPM = msg.tempo #en microseconds per quarter note
                    ticks2seconds = BPM / PPQ  /1000000
            
        m = Melodia(notas, overlap = 0)

        return m

#Unlike music, tempo in MIDI is not given as beats per minute (BPM), but rather in microseconds per quarter note, with a default tempo of 500000 microseconds per quarter note. Given a default 4/4 time signature where a beat is exactly a quarter note, this corresponds to 120 beats per minute.
#In case of different time signatures, the length of a beat depends on the denominator of the time signature. E.g. in 2/2 time signature a beat has a length of a half note, i.e. two quarter notes. Thus the default MIDI tempo of 500000 corresponds to a beat length of 1 second which is 60 BPM.
            

    def sintetizar(self, sr, vibratoDepth = 0.2, vibratoFreq = 1.5, fc = 1500): 
        #vibrato depth: 1 = un semitono hacia arriba y hacia abajo, 2 = 1tono, etc
        #vibDt: tiempo que tarda en llegar a 100% (tiene una curvita de ataque). Esto se setea autoamtico, no le pasé parámetro
        #fc se refiere a la frecuencia de corte de un filtro pasabajos que se le agrega como "de yapa" para simular un poco un cuerpo resonante real

        durTotal = np.sum([nota.dur for nota in self.notas])

        out = np.zeros(round(durTotal* sr))

        for nota in self.notas:
            nota.vibDepth = vibratoDepth
            nota.vibFreq = vibratoFreq
            s = nota.sintetizar3(sr)
            ni = round(nota.ti * sr) 
            nf = ni + round(nota.dur*sr)
            out[ni:nf] += s

        #pasabajos
        sos = CIS_6.signal.butter(1, fc, 'lowpass', fs=sr, output='sos')
        out = CIS_6.signal.sosfilt(sos, out)
        return out

    def sintetizarNoise(self, sr, bw = 2, fc = 1500): #bw es el ancho de los noisebands en semitonos
        out = np.zeros(round(self.dur* sr))
        for nota in self.notas:
            s = nota.sintetizarNoise(sr, bw)
            ni = round(nota.ti * sr) 
            nf = ni + len(s)
            out[ni:nf] += s
        
        #pasabajos para "unificar" el timbre
        sos = CIS_6.signal.butter(1, fc, 'lowpass', fs=sr, output='sos')
        out = CIS_6.signal.sosfilt(sos, out)

        return out

        
    def printInformeScores(self):
        print()
        for i, nota in enumerate(self.notas):
            print(f'--- Nota {i}')
            nota.printInformeScores()
            print('-------------------------\n')
        print('-------------------------\n')
        print(f'Score melodia: {self.score}')



In [16]:
#Recibe el resultado de buscarMejoresCombinacionesMelodia() y exporta un txt con mucha info relevante
import sys
def exportReporteCompleto(clases_melo, outpath, puestos = (0, 10), clases = (3,6)):

    #Parametros usados:
    f0min = f'Parametro de F0MIN usado: {F0MIN}'
    f0max = f'Parametro de F0MIN usado: {F0MAX}'
    n_Mayus = f'Parametro de F0MIN usado: {F0MIN}'
    formula = f'Formula usada: (dist/50)**2 * (2**armonico_i)'   

    file = open(outpath, mode = 'w')
    orig_stdout = sys.stdout #Voy a redirigir los prints al file
    sys.stdout = file

    print(formula)
    print(f0min)
    print(f0max)
    print(n_Mayus)


    clasei, clasef = clases
    puestoi, puestof = puestos

    for clase in range(clasei, clasef+1):
        print(f'--------------------------- CLASE {clase}----------------------')
        print()
        for puesto in range(puestoi, puestof):
            print(f'#############   Puesto  {puesto}   ##############')
            melo = clases_melo[clase][puesto]
            #melo.printInformeScores()

            for i, nota in enumerate(melo.notas):
                print(f' ///// Nota {i}')
                nota.printInformeScores()

                print('//////////////////////////// \n')
            print(f'Score melodia: {melo.score}')

    sys.stdout = orig_stdout
    file.close()


# Generador de combinaciones

Tengo todas las combinaciones de dos cosas: 

Por un lado: Patrones
Por otro: Melodias (trasposicion + armonicos)


In [17]:
#Genera todas las combinaciones posibles de patrones de activacion con entre N_min y N_max canales activos
#frecuencias de los elecs dadas por fes

def generarPatronesActivacion3(fes, nChann_min, nChann_max):

    Elecs = [Elec(fes[i], False, i) for i in range(len(fes))] # todos los elecs
    Combinaciones = generarPatronesActivacion_rec3(0, Elecs, nChann_max,0)  #Todos los patrones posibles

    #Hasta acá en realidad trabajé con "Patrones" truchos para no usar la estructura de datos y acelerar un poco la recursión que puede ser gigante.
    #Ahora sí, paso estos a verdaderos Patrones con estructura
    #y descarto los patrones con menos de nChann_min
    
    Patrones = []
    for combinacion_elecs in Combinaciones:
        P = Patron(combinacion_elecs)
        if(P.N >= nChann_min): Patrones.append(P) 
        
    return Patrones

from copy import deepcopy
def generarPatronesActivacion_rec3(i, elecs, nChann_max, nPrendidos): #Parte recursiva de la funcion de arriba
   
    # i = electrodo del cual voy a ramificar en esta iteracion
    if(i >= len(elecs) or nPrendidos >= nChann_max): return [elecs] #devuelvo como lista de lista porque en el otro return hago append, y quedaria [posible1] + [posible2] + [posible3] + ...

    patronA = [Elec(elec.freq, elec.on, elec.i) for elec in elecs] #deepcopy(elecs)  deepcopy es muy muy lento. Por eso evito usarlo (hace que tarde más de 10 veces más que ese for que puse)
    patronB = [Elec(elec.freq, elec.on, elec.i) for elec in elecs] #deepcopy(elecs)

    patronA[i].on = True
    patronB[i].on = False

    return generarPatronesActivacion_rec3(i+1, patronA, nChann_max, nPrendidos+1) + generarPatronesActivacion_rec3(i+1, patronB, nChann_max, nPrendidos)

In [18]:
F0MIN = 100
F0MAX = 3000
#Recibe una melodia y busca que traspocision es la que generaría menor distorsión total
def generarTrasposicionesMelodia(melodia, f0min = F0MIN, f0max = F0MAX):

    #Parto de 250Hz porque si, y voy subiendo de a step
    step = 2**(1/24) #cuarto de tono. 50 cents

    #Busco la nota más grave de la melodía, y transporto la melodía para que esa nota sea fbase
    f_grave, f_aguda = melodia.notasExtremas()
    melodia.transportar(f0min/f_grave)

    #actualizo para las nuevas notas extremas (que son estas mismas transportaradas)
    f_grave, f_aguda = melodia.notasExtremas()

    melodiasTransportadas = []
    #from copy import deepcopy no lo uso porque es LENTO
    while f_grave < f0max and f_aguda < f0max:
        
        #melodiasTransportadas.append(deepcopy(melodia)) deepcopy es muy lento. Lo hago manual mejor
        melodiasTransportadas.append(melodia.copia())


        melodia.transportar(step)
        f_grave *= step
        f_aguda *= step
        
    return melodiasTransportadas #Devuelvo todas 

In [19]:
# genera todos los subespectros posibles de una diente de sierra con n entre nmin y nmax parciales hasta el armonico 8 (o N_Mayus) inclusive
# (hasta 256 espectros ) --> 256 copias

N_Mayus = 12
def generarEspectros(n_min = 3, n_max = 6):

    #Tomo la saw como base hasta armonico 8
    sawAmps = [1/k for k in range(1,N_Mayus+1)]
    espectrosPosibles = generarEspectrosMelodia_rec(0, sawAmps, N_Mayus,n_min)
    out = []
    for amps in espectrosPosibles:

        #reviso que no estén prendido solo los pares (8va arriba):
        #armonicos pares: amps[1], amps[3], amps[5]...
        if (all(amps[i] == 0 for i in range(0, N_Mayus, 2))): continue

        #reviso que no estén prendido solo los multiplos de 3 (8va + 5ta arriba):
        #armonicos 3:  amsp[2], amps[5], amps[8]...
        if (all(amps[i] == 0 and amps[i+1]== 0 for i in range(0, N_Mayus - N_Mayus%3, 3))): continue
        
        #reviso que haya menos de n_max canales. n_min ya se cumple en la recursion
        n = 0
        for a in amps: 
            if (a!=0): n+=1
        if(n<=n_max): out.append(amps)
        if(n<=n_min): Exception('Error al generar espectros!')
    
    return out 
    

#Parte recursiva. Misma idea que para patrones
#n: cuantos activados ahora
#i: cual voy a desactivar en este paso
#n_min: Cuando n llega acá dejo de desactivar
def generarEspectrosMelodia_rec(i, amps, n, n_min):

    # i = electrodo del cual voy a ramificar en esta iteracion
    if(i >= N_Mayus or n<=n_min): return [amps] #devuelvo como lista de lista porque en el otro return hago append, y quedaria [posible1] + [posible2] + [posible3] + ...

    ampsA = amps.copy()
    ampsB = amps.copy()

    #ampsA[i] = ampsA[i] nada
    ampsB[i] = 0

    return generarEspectrosMelodia_rec(i+1, ampsA, n, n_min) + generarEspectrosMelodia_rec(i+1,ampsB, n-1, n_min)

#genera copias de la melodia con todos los "sub"espectros  posibles  una diente de sierra (las creadas arriba)
def generarCombinacionesEspectrosMelodia(melodia, espectros):

    melodiasPosibles = []

    for amps in espectros: 
        #copio y asigno
        m = melodia.copia()
        m.asignarAmps(amps)
        for nota in m.notas: nota.condensarParciales() #quito amplitudes y parciales en 0 
        melodiasPosibles.append(m)

    return melodiasPosibles


# Busqueda mejor match

In [20]:
####################
# Escalo para melodías

#Recibe una melodia y busca que traspocision, espectro, y patrón es el que generaría menor distorsión total
#Devuelve una lista con todos los patrones ordenados por score, y particionados según N electrodos
def buscarMejoresCombinacionesMelodia(freqsElectrodos ,melodia, N_min, N_max,max_di):
    
    #ojo importante: la melodia tiene que tener ya los parciales asignados. O sea, si la leo con midi, pasarle al menos la amplitud de 8 armonicos al crearla. asi se asignan los parciales ahi. eso es porque no creo nuevas notas en el paso de generar espectros sino que solo reasigno las amps

    #1a) enero las trasposiciones posibles
    melodiasTransportadas =  generarTrasposicionesMelodia(melodia) #aprox 150
    print(f'Cantidad de melodias transportadas generadas: ', len(melodiasTransportadas))

    #1b) Por cada una genero los espectros posibles

    espectrosPosibles = generarEspectros(N_min, N_max)
    print(f'Cantidad de espectros a asignar: ', len(espectrosPosibles))
    print(f'(Usando N_Mayus = {N_Mayus} , F0MIN = {F0MIN}, F0MAX = {F0MAX}), ')

    melodiasTransportadasEspectros = []
    for m in melodiasTransportadas:
        melodiasTransportadasEspectros += generarCombinacionesEspectrosMelodia(m, espectrosPosibles)  #(append de listas) 219 combinaciones posibles por cada una

    
    print(f'Cantidad de melodias transportadas y con espectro diferente generadas: ', len(melodiasTransportadasEspectros))

    #Ahora tengo todas las variaciones de melodias posibles. Del orden de 20.000 posibles

    #1c) particiono melodias segun n de armonicos 
    #asi accedo segun N clases (y no segun n-1)
    melodias_part = [[] for _ in range(N_max+1) ]
    for m in melodiasTransportadasEspectros:
        N = len(m.notas[0].amps)
        melodias_part[N].append(m)

    #Hasta acá, deberia tardar entre 2-5 s

    #2a)Genero todos los patrones posibles
    patronesPosibles = generarPatronesActivacion3(freqsElectrodos, N_min, N_max) #genero todas las combianciones posibles acá. Es mejor que generarlos cada vez como hacia antes 
        
    print(f'Cantidad de patrones de activacion posibles generados: ', len(patronesPosibles))


    #2b)Particiono Patrones según N canales
    Patrones_part = [[] for _ in range(N_max+1) ] # ESTA NOTACION--> [[]] *(N_max+1) NO VA, porque todas las listas internas resultan ser copias unas de otras. ej, si en la primera hago append, se hace append en todas. Si, chotisimo..
    for patron in patronesPosibles:
        Patrones_part[patron.N].append(patron)
    #Ahora Patrones_part[3] tiene los patrones con 3 canales, y así

    #Hasta acá, deberia tardar entre 5-10 s para pablo pasea

    #ESTO es lo que tarda, pegando el salto de MUCHO TIEMPO en esto que sigue (2hs+ para UNA nota!! antes no tardaba tanto... en parte por usar elec medel (pero use un cochlaer tam...) y por no complicarme con los espectros como si hice aca, lpm)
    #Para 6 notas: 14hs

    # Puntúo todos los patrones posibles CON EL MISMO N para cada melodia
    for N in range(N_min, N_max+1):
        mi = 0
        print(f'Asignando patrones y scores para clase N = {N} (Patrones: {len(Patrones_part[N])} , Melodias: {len(melodias_part[N])})')

        for m in melodias_part[N]:
            puntuarPatronesPosiblesMelodia2(m, Patrones_part[N], max_di)
            
            if(mi % 100 == 0): #Es muy lento el print... necesito hacer esto para acelerar un poco
                print(f'Asignando mejor patron y scores a melodia: {mi} \r', end = '')
            mi += 1
        print()

    #y ordenar ya es rápido

    #Ordeno de menor a mayor error cada clase de N electrodos
    for N in range(N_min, N_max+1):
        melodias_part[N].sort(reverse =False, key = lambda m: m.score)

    
    return melodias_part  #ya puntuadas, ordenadas y particionadas por clase
#Los de las particiones esta bueno tenerlo aca por las funciones generadoras de todas las combinaciones posibles, que si hiciera esta fun solo para un N, se llamarian muchas veces. creo

#ESTO ES LO QUE TARDA!! Son Muchas melodias, y muchos patrones de electrodos!!
def puntuarPatronesPosiblesMelodia2(melodia, patrones, max_di):

    melodia.score = 0
    for nota in melodia.notas: #acá es lo tardado: Para cada nota, de cada melodia posible, recorre todos los patrones!! 
        for patron in patrones:
            #Le asigno puntaje a cada patron para esta nota
            calcularScoresCuadraticos3(nota,  
                                    patron,
                                    max_di)

        #le asigno el mejor patron a la nota. En la siguiente nota los puntajes de patrones se reinician
        mejorpatronnota = SeleccionarMejorScore_particionado(patrones)
        if(mejorpatronnota is not None):
            nota.mejorPatron = mejorpatronnota
            nota.score = mejorpatronnota.score
            melodia.score += nota.score
        else:
            nota.score = np.inf
            melodia.score = np.inf
            return

    return 

In [21]:
def cents(f1,f2):
    return 1200*log2(f1/f2)


###############################################################################


#Asigno los errores cuadraticos a cada patron de activación de la lista PARA UNA NOTA
#Lo que hago es puntuar que tan bien aproxima ese patron a los parciales de esa nota y decidir el error en base a eso
#Si esto se puede acelerar, genial
def calcularScoresCuadraticos3(nota, patron, max_di):
    score = 0
    for elec in patron.elecs:
        masCercana, armonico_i, abs_dist = elec.buscarFreqMascercana(nota)
        if(abs_dist > max_di):
            score = np.inf 
            break
        else: 
            score += scoreRelacion4(abs_dist) * nota.dur
            #elec.fo = masCercana #todo Creo que no hace falta ver que dos electrodos no repitan frecuencia más cercana...pero puede pasar si la fundamental es de baja frecuencia. Ej f0 = 100hz, armonicos 8 y 9 por ahi le pegan juntos a 850hz? aunque no creo
    patron.score = score
    return 

#Y si en vez de recorrer los electrodos, recorro los parciales de la nota y les voy asignando electrodos? Rta: Eso no lo hago porque puedo dejar parciales afuera, y sería complicado ver cuales no asignar


#Mayor Score = Mayor Error
#todo priorizar stretched intervals sobre narrowed
#dist es el intervalo en cents
#armonico i arranca de 0
def scoreRelacion(f, dist, amp, armonico_i):
    return (dist/50)**2 * (2**armonico_i)   #todo revisaaaar

#todo y si "i" va en el exponente de dist?
#todo tener en cuenta amp
#todo tambien tener en cuenta el Nchann

#Mayor Score = Mayor correlación = Menor Error 
def scoreRelacionB(f, dist, amp, armonico_i):
    return 1/(dist-1)**2

#Mayor Score = Mayor correlación = Menor Error 
def scoreRelacionC(f, dist, amp, armonico_i):
    return abs(1/np.ln(dist))

def scoreRelacion4(dist):
    return (dist)**2
    

#Idea: Un score que sume puntos en vez de sumar error
     


#Toma una lista de scores y selecciona el que tiene menos error con la cantidad de canales activos indicados
#Si no encuentra con esa cantidad, busca el que tenga n-1 canales, y así.
def SeleccionarMejorScore(patrones, nChann):
    if(nChann == 0): return Patron([]) #score infinito
    
    min_err = np.inf
    best_pattern = None
    for patron in patrones:
        if patron.score < min_err and patron.NcanalesActivos() == nChann:
            min_err = patron.score
            best_pattern = patron
    if(best_pattern == None):  return SeleccionarMejorScore(patrones, nChann-1) #no encontró patrón con esa cantidad de canales pedida --> Busca con menos  
    return best_pattern

#toma una lista de patrones y simplemente devuelve el de menor score
def SeleccionarMejorScore_particionado(patrones):
    min_err = np.inf
    best_pattern = None
    for patron in patrones:
        if patron.score < min_err:
            min_err = patron.score
            best_pattern = patron
    return best_pattern


#Todo si hay un solo canal que sea la fundamental

#Todo penalidad por no representar un armonico
#todo nmin nmax
#todo espectros

#todo PRECALCULAR ARRAY LOCO

#Busca la frecuencia encendida más cercana a una dada en un patrón de activación
def buscarParMascercanoFreq(f, patron):
    min_index = -1
    min_interval = 3000
    for i in range(len(patron)):
        freq, on = patron[i].freq, patron[i].on
        interval = np.abs(cents(f,freq))
        if on and interval < min_interval:
            min_interval = interval
            min_index = i
    return patron[min_index]



# Síntesis Mejor Match

In [22]:
class Elec_Array(CIS_6.Elec_Array): #extiende CIS_6


    #recibe una melodia , donde cada nota ya tiene asignado un patron de electrodos que le corresponde, y genera la señal que estimula esos electrodos
    # Devuelve cuatro copias: una donde sintetiza como se escucharian esas frecuencias de la melo original con sinusoides, otra con ruido,
    # otra donde sintetiza para que lo escuche el implantado con senos y otra con ruido

    def generarSenialesMelodia(self, melodia, sr = 44100, vibDepth = 0.1, vibFreq =3, bw = 2):
        

        #Para saber como suena la melodia elegida (8va, armonicos, etc). Esto es re para el oyente. Uso el sr comun
        origSin = melodia.sintetizar(sr = sr, vibratoDepth = vibDepth, vibratoFreq = vibFreq) #ya usa sins de base
        origNoise = melodia.sintetizarNoise(sr = sr, bw = bw)

        #Para uno hacerse una idea como deberia sonar con el implante. Uso el sr comun.
        #Estimulo las freqs de los electrodos
        #Deberia ser parecida a la original pero con los parciales desafinados y estimulacion mas ancha
        N = round(len(origSin))
        oyente_reasignada_sin = np.zeros(N)
        oyente_reasignada_noise = np.zeros(N)
        
        #Para darle a la persona implantada. Uso el sr del implante.
        #estimulo las freqs de los canales de analisis
        N2 = round(self.SR*melodia.dur)
        CI_reasignada_sin = np.zeros(N2)
        CI_reasignada_noise = np.zeros(N2)
        
        print('\nHaciendo oyente reasignada sin y noise')            
        for nota in melodia.notas:
            
            CIS_6.D = 0.01
            sin1 = self.sintesisPatronElecs(nota, sr)  
            
            CIS_6.D = 0.05
            print('\nHaciendo oyente reasignada noise')
            noise1 =  self.sintesisPatronElecs(nota, sr)

            ni = round(nota.ti*sr)
            nf = ni + len(sin1)            
            oyente_reasignada_sin[ni:nf] += sin1
            oyente_reasignada_noise[ni:nf] += noise1
        

        print('\nHaciendo CI  reasignada sin y noise')
        for nota in melodia.notas:

            notaCopy = nota.copia()
            self.compensarPreenfasisNota(notaCopy, N2)


            sin2 = self.generarSenialPatron2(notaCopy, self.SR)
            noise2 =  self.sintesisPatronAnalisis(notaCopy)

            ni = round(notaCopy.ti*self.SR)
            nf = ni + len(sin2)            
            CI_reasignada_sin[ni:nf] += sin2
            CI_reasignada_noise[ni:nf] += noise2
 
        return origSin, origNoise, oyente_reasignada_sin, oyente_reasignada_noise , CI_reasignada_sin, CI_reasignada_noise

    # Genero las freqs de los electrodos 
    def sintesisPatronElecs(self, nota, sr):
        noise = CIS_6.wnoise_sig(round(nota.dur*sr))
        indices_elecs_activos = [elec.i for elec in nota.mejorPatron.elecs]
        outs = CIS_6.filter_bank_sintesis2(noise, self, indices_elecs_activos, sr = sr)
        for out, amp in zip(outs, nota.amps): out *= amp #escalo por la amplitud de cada parcial
        return np.sum(outs, axis = 0)

    # genero señales en los canales de analisis 
    def sintesisPatronAnalisis(self, nota):
        noise = CIS_6.wnoise_sig(round(nota.dur*self.SR))
        indices_elecs_activos = [elec.i for elec in nota.mejorPatron.elecs]
        outs = CIS_6.filter_bank_analisis(noise, self, orden = 6)
        outs= [outs[i] for i in indices_elecs_activos] #selecciono solo los canales que necesito
        for i, out in enumerate(outs): out *= nota.amps[i] #escalo por la amplitud de cada parcial
        return np.sum(outs, axis = 0)

    #Recibe una nota con patron de activacion y genera una señal que en el implante activaría solo ese patron (usa sinusoides)
    def generarSenialPatron2(self, nota, sr = None):

        out = np.zeros( round(nota.dur*sr))

        #Frecuencias que necesito estimular (la de analisis de cada electrodo),
        freqs_patron = [self.freqsAnalisis[elec.i] for elec in nota.mejorPatron.elecs] 
        
        #creo una nueva nota donde los parciales son lestas freqs para sintetizarla
        nota2 = Nota(nota.freq, nota.amps, nota.dur,  freqs_patron, nota.amp, nota.vibDepth, nota.vibFreq, nota.envinfo)
        out = nota2.sintetizar3(sr)
        return out

    def compensarPreenfasisNota(self, nota, nsamps):
        from scipy import signal
        from scipy.fft import fft

        #Importante: A las señales que les voy a dar al implante, aplico el filtro de preenfasis inverso!
        # Calculo |H(w)| de este filtro de Pre-énfasis
        b, a = signal.butter(1, 1200,btype = 'highpass', fs = self.SR , output='ba')
        d = [1] + [0]*(nsamps-1) #delta
        h = signal.lfilter(b, a, d) #ir
        H = fft(h)

        #Ahora busco los indices de las freqs de cada parcial
        df_bin = nsamps/self.SR
        for i in  range(len(nota.amps)):
            parcial = nota.parciales[i]
            f_i = round(parcial/df_bin) #saco el indice de ese parcial en H
            nota.amps[i] *= 1/abs(H[f_i]) #compenso por la amplitud del filtro

        return

    def generarSenialMejorMatchMelodia(self, melodia, sr, N_chann, max_di):
        combinaciones = buscarMejoresCombinacionesMelodia(self.freqsElectrodos, melodia , N_chann, N_chann, max_di)
        return self.generarSenialesMelodia(combinaciones[N_chann][0], sr = sr)
            
    def generarTodasSeniales(self, melodia, sr):
        origSin, origNoise, oyente_reasignada_sin, oyente_reasignada_noise, CI_reasignada_sin, CI_reasignada_noise = self.generarSenialesMelodia(melodia, sr = sr )
        CIS_6.MODE = CIS_6.ACE
        outSinReasignacion = self.CIS(origSin, sr)

        print('Melo original (sin)')
        display(Audio(origSin, rate = sr))

        print('Melo original (Noise)')
        display(Audio(origNoise, rate = sr))

        print('Sin reasignación - Percepción estimada')
        display(Audio(outSinReasignacion, rate = CIS_6.OUT_SR))

        print('Con Reasignación - Percepcion estimada (Narrow)')
        display(Audio(oyente_reasignada_sin, rate = sr))

        print('Con Reasignación - Percepcion estimada (Noise)')
        display(Audio(oyente_reasignada_noise, rate = sr))

        print('Con Reasignación - Melo para el implante (sin)')
        display(Audio(CI_reasignada_sin, rate = self.SR))

        print('Con Rasignación - Melo para el implante (Noise)')
        display(Audio(CI_reasignada_noise, rate = self.SR))

        return
        

#Devuelve, en orden, los patrones de electrodos que más se correspoden a una serie armónica
#Problema dificil... usar correlación?
    def buscarPatronesArmonicos(self):
        return 

from IPython.display import Audio
#Otro método: Busco representar intervalos entre dos notas de la manera más fiel, nada más

In [23]:

#El de Francisco, Flex28
GENERIC_MEDEL3 = Elec_Array(N=12, xi = 28, dx = 23.1/11, type = CIS_6.LW) 
GENERIC_MEDEL3.freqsAnalisis = [120,235,384,579,836, 1175,1624,2222,3019,4084,5507,7410] #Logarithmic FS 70-8500
GENERIC_MEDEL3.name = 'Medel Francisco Opus 2 FLEX28'
GENERIC_MEDEL3.manufacturer = CIS_6.MEDEL
GENERIC_MEDEL3.channel1_lowcut = 70 #FSP = 70Hz, HDCIS = 250Hz
GENERIC_MEDEL3.setearParametros()


#AXEL OIDO DERECHO
Cochlear_Axel_Der = Elec_Array(N=22, xi= 20, dx = 0.75, type = CIS_6.LW, name = 'Axel Derecho CI24M - Nucleus 24 199x') #xi puede estar mal
Cochlear_Axel_Der.freqsAnalisis = CIS_6.freqTable22
Cochlear_Axel_Der.manufacturer = CIS_6.COCHLEAR
Cochlear_Axel_Der.setearParametros() #todo rehacer esta funcion de inicializacion para que reciba todos los params de arriba de una... y separar elec array de procesador..

In [24]:
#Esto es necesario porque guardé muchos resultados de melodias y patrones y luego actualice las clases, y entonces al cargar los resultados tengo que recargar los metodos de clase de nota y de melodias
def actualizarInstancias(melodias):

    for melodia in melodias:
        for nota in melodia.notas:
            nota.__class__ = Nota
            if(nota.mejorPatron is not None): nota.mejorPatron.__class__ = Patron
        melodia.__class__ =  Melodia

def actualizarInstancia(melodia):
    for nota in melodia.notas:
            nota.__class__ = Nota
    melodia.__class__ =  Melodia

def actualizarInstanciaClasesN(clases):
    for claseN in clases:
        actualizarInstancias(claseN)
        